In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from helper import get_data, split_data, get_batches

In [3]:
# MISSION:
# Learn if the sum of two integers are higher than 10.

## GRAPH

In [4]:
# HYPERPARAMTERS
batch_size = 50
epochs = 10

In [5]:
# MODEL
x = tf.placeholder(tf.float32, [None, 2])

In [6]:
# Hidden layer:
W_h = tf.Variable(tf.truncated_normal([2, 1], stddev=0.05))
b_h = tf.Variable(tf.random_normal([1]))

hidden = tf.add(tf.matmul(x, W_h), b_h)

In [7]:
W_o = tf.Variable(tf.truncated_normal([1, 2], stddev=0.05))
b_o = tf.Variable(tf.random_normal([1]))

logits = tf.add(tf.matmul(hidden, W_o), b_o)
output = tf.nn.softmax(logits)

In [8]:
y = tf.placeholder(tf.float32, [None, 2])

In [9]:
# Training
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

In [10]:
# Accuracy
correct_pred = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## DATA 

In [11]:
# inputs: two integers
# targets: [0, 1] if the sum is higher than 10. [1, 0] if the sum is lower than 10.

inputs, targets = get_data(max_int=10, size=10000)

In [12]:
# preprocessing: normalize inputs to be between -1 and 1.
inputs = (inputs-5)/5

In [13]:
# split train and test data
train_inputs, test_inputs, train_targets, test_targets = split_data(inputs, targets)

## SESSION

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # TRAINING
    for epoch in range(epochs):

        batch_x, batch_y = get_batches(train_inputs, train_targets, batch_size)

        for batch in range(train_inputs.shape[0]//batch_size):

            sess.run(optimizer, feed_dict={
                x: batch_x[batch],
                y: batch_y[batch]
                })

            train_loss = sess.run(cost, feed_dict={
                x: batch_x[batch],
                y: batch_y[batch]
                })

            if batch == 0:
                print('Epoch {:>2}, Batch {:3} - '
                    'Training Loss: {:>10.4f}'.format(
                        epoch + 1, 
                        batch + 1,
                        train_loss))
    
    # TESTING
    test_accuracy = sess.run(accuracy, feed_dict={
        x: test_inputs,
        y: test_targets
        })
    print('\nTesting Accuracy: {}'.format(test_accuracy))


    final_test = sess.run(output, feed_dict={
        x: (np.array([[5, 3], [7, 6], [10, 10]])-5)/5
        })
    print('\nFinal test (5 + 3), (7 + 6), (10 + 10):')
    print(final_test)

    print("\nHidden layer weights and bias:")
    print(sess.run(W_h))
    print(sess.run(b_h))
    print("Output layer weights and bias:")
    print(sess.run(W_o))
    print(sess.run(b_o))

Epoch  1, Batch   1 - Training Loss:     0.7004
Epoch  2, Batch   1 - Training Loss:     0.6842
Epoch  3, Batch   1 - Training Loss:     0.6375
Epoch  4, Batch   1 - Training Loss:     0.5609
Epoch  5, Batch   1 - Training Loss:     0.5397
Epoch  6, Batch   1 - Training Loss:     0.3738
Epoch  7, Batch   1 - Training Loss:     0.2551
Epoch  8, Batch   1 - Training Loss:     0.2203
Epoch  9, Batch   1 - Training Loss:     0.2034
Epoch 10, Batch   1 - Training Loss:     0.1641

Testing Accuracy: 1.0

Final test (5 + 3), (7 + 6), (10 + 10):
[[  8.93008471e-01   1.06991500e-01]
 [  1.24206737e-01   8.75793278e-01]
 [  4.71629581e-04   9.99528408e-01]]

Hidden layer weights and bias:
[[-1.64704931]
 [-1.64078069]]
[ 0.19933932]
Output layer weights and bias:
[[ 1.21974468 -1.26005733]]
[-0.2669307]
